# Базовые понятия SQL

In [2]:
!pip install pandasql

  Using cached pandasql-0.7.3-py3-none-any.whl


In [3]:
import pandas as pd
import numpy as np
from sklearn import datasets
import datetime as dt

from pandasql import sqldf

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Описание

Ноутбук формализует основные концепты языка SQL. Приведены основные практики с примерами задач, где они могут использоваться 

# Концепты

## Порядок выполнения команд - execution order

Python - это императивный язык программирования, в нем мы описываем **как** сделать то, что нам нужно. А SQL это декларативный язык, в нем мы описываем **что** хотим получить. Разница между этими двумя подходами проявляется в последновательности исполнения комманд. В SQL они выполняются не в порядке их ввода (как в питоне), а в строго заданной последовательности:

[Ресурс](https://techrocks.ru/2021/03/05/order-of-sql-operations/)

1. FROM (выбор таблицы) / JOIN (комбинация с подходящими по условию данными из второй таблицы)
2. WHERE (фильтрация строк)
3. GROUP BY (агрегирование данных)
4. HAVING (фильтрация агрегированных данных)
5. SELECT (возврат результирующего датасета) / CASE (if-else выражения)
6. DISTINCT
7. UNION (объединение )
8. ORDER BY (сортировка)
9. LIMIT, TOP OFFSET

Как правило, запрос исполняется именно в такой последовательности.


Введение в синтаксис будет рассмотрено на примере открытой базы данных, предназначенной специально для практики SQL. Чтобы твое обучение прошло максимально эффективно, открой ссылку ниже в новой вкладке и сразу запускай приведенные примеры, это позволит тебе лучше закрепить материал и самостоятельно поработать с синтаксисом.

https://www.w3schools.com/sql/trysql.asp?filename=trysql_op_in

После перехода по ссылке можно будет увидеть сам редактор запросов и вывод данных в центральной части экрана, список таблиц базы данных находится в правой части.

Разберем структуру. Для удобства текущий изучаемый элемент в запроса выделяется CAPS'ом.

SELECT, FROM

SELECT, FROM — обязательные элементы запроса, которые определяют выбранные столбцы, их порядок и источник данных.

Выбрать все (обозначается как *) из таблицы Customers:

**SELECT * FROM Customers**

Выбрать столбцы CustomerID, CustomerName из таблицы Customers:

**SELECT CustomerID, CustomerName FROM Customers**

WHERE

WHERE — необязательный элемент запроса, который используется, когда нужно отфильтровать данные по нужному условию. Очень часто внутри элемента where используются IN / NOT IN для фильтрации столбца по нескольким значениям, AND / OR для фильтрации таблицы по нескольким столбцам.

Фильтрация по одному условию и одному значению:

select * from Customers
WHERE City = 'London'

Фильтрация по одному условию и нескольким значениям с применением IN (включение) или NOT IN (исключение):

select * from Customers
where City IN ('London', 'Berlin')

select * from Customers
where City NOT IN ('Madrid', 'Berlin','Bern')

Фильтрация по нескольким условиям с применением AND (выполняются все условия) или OR (выполняется хотя бы одно условие) и нескольким значениям:

select * from Customers
where Country = 'Germany' AND City not in ('Berlin', 'Aachen') AND CustomerID > 15

select * from Customers
where City in ('London', 'Berlin') OR CustomerID > 4


GROUP BY

GROUP BY — необязательный элемент запроса, с помощью которого можно задать агрегацию по нужному столбцу (например, если нужно узнать какое количество клиентов живет в каждом из городов).

При использовании GROUP BY обязательно:

перечень столбцов, по которым делается разрез, был одинаковым внутри SELECT и внутри GROUP BY,
агрегатные функции (SUM, AVG, COUNT, MAX, MIN) должны быть также указаны внутри SELECT с указанием столбца, к которому такая функция применяется.

Группировка количества клиентов по городу:

select City, count(CustomerID) from Customers
GROUP BY City

Группировка количества клиентов по стране и городу:

select Country, City, count(CustomerID) from Customers
GROUP BY Country, City

Группировка продаж по ID товара с разными агрегатными функциями: количество заказов с данным товаром и количество проданных штук товара:


select ProductID, COUNT(OrderID), SUM(Quantity) from OrderDetails
GROUP BY ProductID

Группировка продаж с фильтрацией исходной таблицы. В данном случае на выходе будет таблица с количеством клиентов по городам Германии:


select City, count(CustomerID) from Customers
WHERE Country = 'Germany'
GROUP BY City

Переименование столбца с агрегацией с помощью оператора AS. По умолчанию название столбца с агрегацией равно примененной агрегатной функции, что далее может быть не очень удобно для восприятия.

select City, count(CustomerID) AS Number_of_clients from Customers
group by City

HAVING

HAVING — необязательный элемент запроса, который отвечает за фильтрацию на уровне сгруппированных данных (по сути, WHERE, но только на уровень выше).

Фильтрация агрегированной таблицы с количеством клиентов по городам, в данном случае оставляем в выгрузке только те города, в которых не менее 5 клиентов:


select City, count(CustomerID) from Customers
group by City
HAVING count(CustomerID) >= 5 


В случае с переименованным столбцом внутри HAVING можно указать как и саму агрегирующую конструкцию count(CustomerID), так и новое название столбца number_of_clients:


select City, count(CustomerID) as number_of_clients from Customers
group by City
HAVING number_of_clients >= 5

Пример запроса, содержащего WHERE и HAVING. В данном запросе сначала фильтруется исходная таблица по пользователям, рассчитывается количество клиентов по городам и остаются только те города, где количество клиентов не менее 5:


select City, count(CustomerID) as number_of_clients from Customers
WHERE CustomerName not in ('Around the Horn','Drachenblut Delikatessend')
group by City
HAVING number_of_clients >= 5

ORDER BY

ORDER BY — необязательный элемент запроса, который отвечает за сортировку таблицы.

Простой пример сортировки по одному столбцу. В данном запросе осуществляется сортировка по городу, который указал клиент:


select * from Customers
ORDER BY City

Осуществлять сортировку можно и по нескольким столбцам, в этом случае сортировка происходит по порядку указанных столбцов:


select * from Customers
ORDER BY Country, City

По умолчанию сортировка происходит по возрастанию для чисел и в алфавитном порядке для текстовых значений. Если нужна обратная сортировка, то в конструкции ORDER BY после названия столбца надо добавить DESC:


select * from Customers
order by CustomerID DESC

Обратная сортировка по одному столбцу и сортировка по умолчанию по второму:

select * from Customers
order by Country DESC, City

JOIN

JOIN — необязательный элемент, используется для объединения таблиц по ключу, который присутствует в обеих таблицах. Перед ключом ставится оператор ON.

Запрос, в котором соединяем таблицы Order и Customer по ключу CustomerID, при этом перед названиям столбца ключа добавляется название таблицы через точку:

select * from Orders
JOIN Customers ON Orders.CustomerID = Customers.CustomerID

Нередко может возникать ситуация, когда надо промэппить одну таблицу значениями из другой. В зависимости от задачи, могут использоваться разные типы присоединений. INNER JOIN — пересечение, RIGHT/LEFT JOIN для мэппинга одной таблицы знаениями из другой,


select * from Orders
join Customers on Orders.CustomerID = Customers.CustomerID
where Customers.CustomerID >10

## Примеры

In [9]:
import pandasql as pd_sql

In [10]:
titanic_ds = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
titanic_ds.head()
titanic_ds.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [11]:
pd_sql.sqldf('''
select * from titanic_ds
''')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,None,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,None,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [12]:
pd_sql.sqldf('''
select t_ds.* from titanic_ds as t_ds
'''
)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,None,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,None,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [13]:
pd_sql.sqldf('''
select t_ds.* from titanic_ds as t_ds limit 10
'''
)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,None,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,None,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,None,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,None,C


In [14]:
pd_sql.sqldf('''
select t_ds.name,t_ds.age from titanic_ds as t_ds order by age desc limit 5
''')

,Name,Age
0,"Barkworth, Mr. Algernon Henry Wilson",80.0
1,"Svensson, Mr. Johan",74.0
2,"Goldschmidt, Mr. George B",71.0
3,"Artagaveytia, Mr. Ramon",71.0
4,"Connors, Mr. Patrick",70.5


In [15]:
pd_sql.sqldf('''
select t_ds.passengerId,t_ds.name,t_ds.sex, t_ds.age, t_ds.ticket
from titanic_ds as t_ds
where (t_ds.age < 40 and t_ds.sex = 'male')
or ((t_ds.age > 20 and t_ds.age < 40) and t_ds.sex = 'female')
order by t_ds.age desc
limit 50
''')

,PassengerId,Name,Sex,Age,Ticket
0,14,"Andersson, Mr. Anders Johan",male,39.0,347082
1,401,"Niskanen, Mr. Juha",male,39.0,STON/O 2. 3101289
2,529,"Salonen, Mr. Johan Werner",male,39.0,3101296
3,559,"Taussig, Mrs. Emil (Tillie Mandelbaum)",female,39.0,110413
4,578,"Silvey, Mrs. William Baird (Alice Munger)",female,39.0,13507
5,582,"Thayer, Mrs. John Borland (Marian Longstreth M...",female,39.0,17421
6,611,"Andersson, Mrs. Anders Johan (Alfrida Konstant...",female,39.0,347082
7,706,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,250655
8,796,"Otter, Mr. Richard",male,39.0,28213
9,807,"Andrews, Mr. Thomas Jr",male,39.0,112050


In [16]:
pd_sql.sqldf('''
select t_ds.passengerId,t_ds.pclass,t_ds.name,t_ds.sex, t_ds.age, t_ds.ticket
from titanic_ds as t_ds
where (t_ds.age between 15 and 40)
and
t_ds.pclass in (1,3)
''')

,PassengerId,Pclass,Name,Sex,Age,Ticket
0,1,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599
2,3,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803
4,5,3,"Allen, Mr. William Henry",male,35.0,373450
...,...,...,...,...,...,...
361,885,3,"Sutehall, Mr. Henry Jr",male,25.0,SOTON/OQ 392076
362,886,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,382652
363,888,1,"Graham, Miss. Margaret Edith",female,19.0,112053
364,890,1,"Behr, Mr. Karl Howell",male,26.0,111369


In [17]:
pd_sql.sqldf('''
select count(t_ds.sex) total_passengers, t_ds.sex Gender
from titanic_ds as t_ds
group by t_ds.sex
order by t_ds.sex desc
''')

,total_passengers,Gender
0,577,male
1,314,female


In [18]:
pd_sql.sqldf('''
select t_ds.passengerId,t_ds.pclass,t_ds.name, t_ds.age, 
case
    when (t_ds.age > 55 and t_ds.age <= 130) then 'OLD'
    when (t_ds.age <= 55 and t_ds.age > 38) then 'ADULT'
    when (t_ds.age <= 38 and t_ds.age > 22) then 'YOUNG'
    when (t_ds.age <= 22 and t_ds.age >= 15) then 'TEENAGER'
    when t_ds.age <15 then 'CHILD'
    else 'INVALID'
end as 'AGE_GROUP'
from titanic_ds as t_ds
'''
)

,PassengerId,Pclass,Name,Age,AGE_GROUP
0,1,3,"Braund, Mr. Owen Harris",22.0,TEENAGER
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,YOUNG
2,3,3,"Heikkinen, Miss. Laina",26.0,YOUNG
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,YOUNG
4,5,3,"Allen, Mr. William Henry",35.0,YOUNG
...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",27.0,YOUNG
887,888,1,"Graham, Miss. Margaret Edith",19.0,TEENAGER
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",NaN,INVALID
889,890,1,"Behr, Mr. Karl Howell",26.0,YOUNG


In [19]:
pd_sql.sqldf('''
select count(t_ds.sex) Total_Passengers,avg(t_ds.age) Average_Age, t_ds.sex Gender
from titanic_ds as t_ds
group by t_ds.sex
order by t_ds.sex desc
''')

,Total_Passengers,Average_Age,Gender
0,577,30.726645,male
1,314,27.915709,female
